# Generate tiled region run file

## This notebook is an example: create a copy before running it or you will get merge conflicts!

**NOTE**: Before running this notebook for the first time, make sure you've coregistered your instrument using the *update coregistration parameters* section of  `1_set_up_toffy.ipynb`. This will ensure your FOVs display correctly on the slide.

This notebook allows you to automatically set up large tiled runs of contiguous FOVs. You can specify the dimensions of the tiled image, for example 5x5, and select specific FOVs you wish to exclude from the final tile.

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import json
import os
from skimage.io import imread

from toffy.json_utils import read_json_file, write_json_file, split_run_file
from toffy import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

## 1. Define the data paths

You will first need to define the prefix to use for all of the files associated with this specific TMA. The default is `'example_tiled_region'`, but you should change it to something relevant to your study, such as `'BRCA_TMA_1'`.

Once you have picked your prefix for this specific TMA, you'll need to ensure that all of the necessary files are in the appropriate directory with the correct names

* `region_corners_path`: this file, which contains the top-left corner FOV of each tiled region, should be named `{tiled_region_prefix}_region_corners.json`. Generate this file using MIBIcontrol and give each top-left FOV a descriptive name of the tiled region it defines.
* `slide_path`: this file, which contains the image of your slide, should be named `{tiled_region_prefix}.bmp` or `{tiled_region_prefix}.jpg`. <b>This file can be found on the CAC at: `D:\\Data\\optical-image\\`</b>.

Each of these files should be copied over to `C:\\Users\\Customer.ION\\Documents\\tiled_run_jsons`. More information on the uses and locations of the directories in toffy can be found in the <a href="https://github.com/angelolab/toffy#directory-structure">README</a>.

In [ ]:
# define the prefix for each file
tiled_region_prefix = 'example_tiled_region'

In [ ]:
# user created files
tiled_regions_dir = os.path.join('C:\\Users\\Customer.ION\\Documents\\tiled_run_jsons')
region_corners_path = os.path.join(tiled_regions_dir, '%s_region_corners.json' % tiled_region_prefix)

# change slide_path_ext to '.jpg' if needed
slide_path_ext = '.bmp'
slide_path = os.path.join(tiled_regions_dir, tiled_region_prefix + slide_path_ext)

# the final run file the notebook will generate
tiled_region_path = os.path.join(tiled_regions_dir, "%s_tiled_region_mapping.json" % tiled_region_prefix)

## 2. Generate the FOVs for each region

This cell will prompt you to set the dimensions and randomization parameters of each tiled region. The parameters will be used to generate the FOVs over each tiled region.

In [ ]:
# set the tiling parameters
tiling_params = tiling_utils.set_tiled_region_params(region_corners_path)

# generate the FOVs in each region using tiling_params
tiled_region_fovs = tiling_utils.generate_tiled_region_fov_list(tiling_params)

## 3. Interactive FOV deselection

The grid defining an ROI may cover more area than you wish to tile. This GUI allows you to interactively delete FOVs from *tiled_region_fovs* prior to saving:

<b>NOTE: if you run the widget and do not need to run deselection, click *Save mapping* immediately to generate the final run file with all the FOVs generated.</b>

1. Inspect using Zoom feature.

<div>
    <img src="img/tiled_region_zoom.gif" width="300" height="274"/>
</div>

2. Propose/unpropose FOVs for removal by clicking. Click *Detele selected FOVs* to remove proposed FOVs. <b>This action cannot be undone!</b>

<div>
    <img src="img/tiled_region_selection.gif" width="300" height="274"/>
</div>

4. You can always remove more FOVs even after saving. Just remember to re-save.
5. After you're done, click *Save mapping*. The final run file will be saved to *tiled_region_path*.

In [ ]:
# read the slide image in
slide_data = imread(slide_path)

%matplotlib widget
_ = tiling_utils.tiled_region_interactive_remap(
    tiled_region_fovs,
    tiling_params,
    slide_data,
    tiled_region_path,
    figsize=(7, 7)
)

## 5. Run JSON adjustment

If you would like to break up your tiled file into smaller JSONs containing a specified amount of FOVs in each, you can do that using the code below. 

`file_split` is a list of values detailing how many FOVs to included in each new file, and must sum to the total number of FOVs in the run file.

In [ ]:
file_split = []

split_run_file(tiled_regions_dir, os.path.basename(tiled_region_path), file_split)